# PARCIAL 1 TAM (Teoría de Aprendizaje de Máquina)




## Ej 1 


Dado el modelo de regresion : 

tn = 
𝜙
(
𝑥
𝑛
)
⊤
𝑤
+
𝜂
𝑛
t 
n
​
 =ϕ(x 
n
​
 ) 
⊤
 w+η 
n
​
 , con 
𝜂
𝑛
∼
𝑁
(
0
,
𝜎
2
)
η 
n
​
 ∼N(0,σ 
2
 )


Presentamos la inferencia o problema de optimización para los siguientes casos asumiendo datos i.i.d : 


### MÍNIMOS CUADRADOS (OLS)

### MÍNIMOS CUADRADOS REGULARIZADOS 

### MÁXIMA VEROSIMILITUD 

### MÁXIMO A POSTERIORI (MAP)

### MODELO BAYESIANO LINEAL GAUSSIANO 

### REGRESIÓN RÍGIDA KERNEL

### PROCESOS GAUSIANOS (GP)

### Finalmente discutimos similitudes y diferencias:  

## EJ 2

Presentamos un cuadro comparativo de los siguientes regresores, en donde se discute el modelo matem atico, funcion de costo, estrategia
de optimizacion, relacion con los esquemas b́asicos de regresion discutidos en el anterior ejercicio y tambien su escalabilidad en caso de ser util:


• LinearRegresor
• Lasso
• ElasticNet
• KernelRidge
• SGDRegressor
• BayesianRidge
• Gaussian Process Regressor
• Support Vector Machines Regressor
• RandomForestRegressor

• GradientBoostingRegressor
• XGBoost


## EJ 3

Verificamos en que consiste y el funcionamiento principal de la librer ́ıa RAPIDS. Posteriormente, se elabora una tabla que compara
los metodos e hiperparametros mas relevantes de los regresores mencionados en el anterior ejercicio, indicando su implementacion o el
algoritmo equivalente disponible en RAPIDS.


## EJ 4

Finalmente, se presenta un analisis detallado del conjunto de datos disponible en https://www.kaggle.com/competitions/nfl-big-data-bowl-2026-prediction/overview. Incluyendo la descripcion de las variables de entrada y salida, el objetivo principal del concurso, un analisis
exploratorio basico, los procedimientos de codificacion de valores faltantes y variables categoricas, ası como las estrategias de
ingenierıa de caracterısticas —si aplican—. Finalmente, proporcione todos los detalles necesarios para comprender, interpretar y
analizar adecuadamente el conjunto de datos.
Luego, utilizando un esquema de validacion hold-out con una partici  ́on del 60 % para entrenamiento, 20 % para validacion y 20 %
para evaluacion del rendimiento, compare los regresores mencionados en el punto 2 —considerando sus posibles implementacio-
nes en RAPIDS, segun lo indicado en el punto 3—, empleando el conjunto de datos de entrenamiento disponible en el dataset.
Realice las consideraciones necesarias para la implementacion y sintonizacion de cada modelo mediante optimizacion bayesiana,
con el fin de determinar los hiperparametros mas relevantes de cada algoritmo. Justifique la seleccion de los hiperparametros a
optimizar, ası como la rejilla o los rangos de valores definidos para cada modelo, de acuerdo con las caracterısticas estudiadas
y el score a minimizar. Finalmente, presente los rendimientos promedio obtenidos sobre el conjunto de evaluacion, junto con su
respectiva desviacion est ́andar, empleando las siguientes metricas de desempeno: MAE, MSE, R2 y MAPE

### entorno de desarrollo


primero es necesario configurar nuestro  entorno, comenzando por la importancion de las librerias y dependencias necesarias para este caso, una vez instaladas en el entorno

In [23]:
import numpy as np
import os
import gdown 
import zipfile 
import pandas as pd                         
from scipy import stats
# Set the default style for seaborn
import seaborn as sns
sns.set(style="whitegrid")      


import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import display, Image
import sklearn 
    



es importante configurar matplotlib

In [24]:
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


también, creamos una carpeta para guardar los resultados y otra para la importación de datos

In [29]:
import os
try:
  os.mkdir('results')
  os.mkdir('data')
  print('carpetas creadas en el entorno de trabajo')
except:
  print("Carpeta results ya existe")

carpetas creadas en el entorno de trabajo


finalmente, los warnings

In [30]:
import warnings 
warnings.filterwarnings(action="ignore", message="^internal gelsd")

###  importación y lectura  de los datos: 


#### usaremos gdown para descargar desde el drive personal (evitando conflictos de permiso) y procederemos con zip file para descomprimir el dataset

In [83]:

# Download dataset from personal Google Drive

# https://drive.google.com/file/d/1dqeoLxOrYAf0-JS2sf5sDHxxIRuX8PjG/view?usp=sharing
fileID = "1dqeoLxOrYAf0-JS2sf5sDHxxIRuX8PjG"
url = f"https://drive.google.com/uc?id={fileID}"
output = "data/NFLdataset.zip"
gdown.download(url, output, quiet=False, fuzzy=True, use_cookies=False)
  
!dir

Downloading...
From (original): https://drive.google.com/uc?id=1dqeoLxOrYAf0-JS2sf5sDHxxIRuX8PjG
From (redirected): https://drive.google.com/uc?id=1dqeoLxOrYAf0-JS2sf5sDHxxIRuX8PjG&confirm=t&uuid=f4c6e820-00cb-4d98-bc93-0e42c57d388b
To: a:\wnOs\LR\machineLearningTheoryEnv\TAM\devEnv\MLT_LR\PYDevEnv\devContent\tests\P1\data\NFLdataset.zip
100%|██████████| 107M/107M [00:02<00:00, 46.6MB/s] 


 Volume in drive A is dataOs
 Volume Serial Number is 42ED-9D11

 Directory of a:\wnOs\LR\machineLearningTheoryEnv\TAM\devEnv\MLT_LR\PYDevEnv\devContent\tests\P1

16/10/2025  10:05    <DIR>          .
16/10/2025  09:50    <DIR>          ..
16/10/2025  10:06            10,420 basis.ipynb
16/10/2025  10:06    <DIR>          data
16/10/2025  09:50    <DIR>          docs
14/10/2025  22:56             4,611 README.md
16/10/2025  09:54    <DIR>          results
               2 File(s)         15,031 bytes
               5 Dir(s)  579,924,164,608 bytes free


#### procedemos descomprimiendo el dataset descargado mediante drive usando ZIPFILE

In [85]:
with zipfile.ZipFile("data/NFLdataset.zip", 'r') as zip_ref:
    zip_ref.extractall("data/ExtractedDataset")

''' second version 
import shutil
shutil.unpack_archive("dataset.zip", "ExtractedDataset", "zip")
'''

' second version \nimport shutil\nshutil.unpack_archive("dataset.zip", "ExtractedDataset", "zip")\n'

es importante recoerdar ignorar estas carpetas en nuestro .gitignore 

#### una vez con el dataset extraído procedemos a realizar la lectura del csv usando pandas o cuDF


# END NB
Cristhian Mateo Almeida Gómez



presentado al profesor Andrés Marino Alvarez Meza. 